# Metrics attributes

In this notebook, we will have a closer look at the new `NvRule` `IMetric` attributes.
In particular, we will look at the following API functions:
* `IMetric.metric_type()`
* `IMetric.metric_subtype()`
* `IMetric.rollup_operation()`
* `IMetric.unit()`
* `IMetric.description()`

The first three methods are particularly useful to filter through a list of metrics,
as they return enum values of three different kinds: `IMetric.MetricType_*`,
`IMetric.MetricSubtype_*` and `IMetric.RollupOperation_*`.

The `unit()` function on the other hand provides a convenient way to get the unit of a metric as a string,
whereas `description()` returns a short textual description for hardware metrics.

## Setup

### Import `ncu_report`

Either extent your `PYTHONPATH` environment variable with the `ncu_report` include
directory (usually located within your installation folder at `extras/python`) or
add the include director as a `site` dir.

In [ ]:
import ncu_report
from ncu_report import IMetric

### Load a report file

In [ ]:
path_to_report_file = "../sample_reports/sample_report.ncu-rep"
report = ncu_report.load_report(path_to_report_file)

### Extract a valid action from the `report`

We use the first `IRange` object of the report which contains
a single `IAction` object. This, in turn will give us access to
the `IMetric`s objects that we want to process further.

In [ ]:
action = report.range_by_idx(0).action_by_idx(0)

## Use case 1: Filter metrics based on their attributes

### Convert all metrics to a list

Using list comprehension, we can make iteration over all metrics very simple.

In [ ]:
metrics = [action[name] for name in action.metric_names()]

The simples way to filter metrics is by using their
`metric_type()`:

In [ ]:
print("Throughput metrics:")
for metric in metrics:
    if metric.metric_type() == IMetric.MetricType_THROUGHPUT:
        print(f"  {metric.name()}: {metric.value():.2f}{metric.unit()}")

More advanced types of filtering might make use of
`metric_type()`, `metric_subtype()` and `rollup_operation()`.

We also use `unit()` to get a string representation of the unit
associated with a given metric:

In [ ]:
print("Averaged Counter metrics per second:")
for metric in metrics:
    if metric.metric_type() == IMetric.MetricType_COUNTER and \
       metric.metric_subtype() == IMetric.MetricSubtype_PER_SECOND and \
       metric.rollup_operation() == IMetric.RollupOperation_AVG:
        print(f"  {metric.name()}: {metric.value():,.0f} {metric.unit()}")

## Use case 2: Accumulate metrics from different reports

Suppose we have other reports (or other actions within the same report) and want to combine all the values of a given metric across all reports (actions).

To demonstrate how we might do this using `rollup_operation()` let's first load another action:

In [ ]:
path_to_other_report_file = "../sample_reports/sample_report_2.ncu-rep"
other_report = ncu_report.load_report(path_to_other_report_file)
other_action = other_report.range_by_idx(0).action_by_idx(0) 

We can define a function that takes the name of the metric we want to accumulate as well as a list of all the actions that contain said metric:

In [ ]:
def accumulate_from_actions(metric_name, actions):
    # construct a list of all values of the metric with name 'metric_name'
    values = [action[metric_name].value() for action in actions]

    # query the rollup operation of the given metric
    operation = actions[0][metric_name].rollup_operation()

    # accumulate all values using the correct rollup operation
    if operation == IMetric.RollupOperation_SUM:
        return sum(values)
    elif operation == IMetric.RollupOperation_AVG:
        return sum(values) / len(values)
    elif operation == IMetric.RollupOperation_MIN:
        return min(values)
    elif operation == IMetric.RollupOperation_MAX:
        return max(values)
    else:
        raise RuntimeError("Cannot accumulate metrics when rollup_operation() == None")

Let's pick the name of a metric we want to look at. (You can comment in/out different names and compare the results.)

In [ ]:
metric_name = "sm__throughput.avg.pct_of_peak_sustained_elapsed"
# metric_name = "gpu__time_duration.sum"
# metric_name = "profiler__replayer_bytes_mem_accessible.min"
# metric_name = "profiler__replayer_bytes_mem_accessible.max"
# metric_name = "sm__maximum_warps_per_active_cycle_pct" # no rollup operation defined

Next, we unpack the metrics, query some of their properties and calculate the accumulated value:

In [ ]:
metric = action[metric_name]
other_metric = other_action[metric_name]

description = metric.description()
unit = metric.unit()
value = metric.value()
other_value = other_metric.value()
accumulated_value = accumulate_from_actions(metric_name, [action, other_action])

Let's look at a summary of what we have found:

In [ ]:
print(f"{metric_name}:")
print(f"  Description: {description}")
print(f"  First value: {value:.2f}{unit}")
print(f"  Second value: {other_value:.2f}{unit}")
print(f"  Accumulated value: {accumulated_value:.2f}{unit}")